# 3.3.3.2 Realistic simulation setup
Obtain the centroid for each chessboard 

In [240]:
vscode = 1


In [241]:
if(vscode == 0):
    # for vscode
    # for jupyter notebook
    from mpl_toolkits.mplot3d import axes3d
    import matplotlib.pyplot as plt

    %matplotlib notebook
elif(vscode == 1):
    %matplotlib qt
    

In [242]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
 
from typing import Sequence
from calib_lib import *
from matplotlib.collections import PatchCollection      # pathcoletion for multiple patches
import matplotlib.cm as cm                              # for colormaps
from matplotlib.patches import Rectangle                # for rectangle representation
from matplotlib.patches import Ellipse                  # for ellipse representation   
from pixelization_lib import *   
#from ellipse import LsqEllipse                          # pip install lsq-ellipse


DECIMALS = 2            # how many decimal places to use in print


In [243]:
import logging 
import numpy as np
from skimage.measure import EllipseModel
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt

In [244]:
F = 16                                       # focal length( in mm )
gridSize = 1024
subpixel_grid = 128
image_size = np.array([1936,1216])               # sensor size(in pixels)
#image_size = np.array([11.345,7.126])
pixel_width = 5.86e-3                       # pixel size in mm
PX= image_size[0]/2.0                       # principal point_i x-coordinate
PY= image_size[1]/2.0                       # principal point_i y-coordinate
IMAGE_HEIGTH = image_size[1]
IMAGE_WIDTH = image_size[0]
THETA_X = 0                                 # roll angle
THETA_Y = 0                                 # pitch angle
THETA_Z = 0                                 # yaw angle

# camera Right
THETA_X_R = 0                                 # roll angle
THETA_Y_R = 0                                 # pitch angle
THETA_Z_R= 0                                 # yaw angle
# camera Left
THETA_X_L = 0                                 # roll angle
THETA_Y_L = 0                                 # pitch angle
THETA_Z_L= 0                                 # yaw angle

C_L = np.array([0,0,0])                     # camera centre
C_R = np.array([500,0,0])

chess_dimx,chess_dimy = (12,8)
chess_sq_size = 44

radius = 5

In [245]:
iterations = 5
meters = 4

In [246]:
xrange_5 = np.linspace(-600,1100,6)
yrange_5 = np.linspace(-550,550,6)
zrange_5 = ([4000])

In [247]:
print('world_pts_arr_4_'+str(iterations)+'_px.npy')

world_pts_arr_2_5_px.npy


In [248]:

world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_1024_point_chess(xrange_5,yrange_5,zrange_5)


np.save('world_pts_arr_'+str(meters)+'_'+str(iterations)+'_px.npy',world_pts_arr)
np.save('x_zhang_R_'+str(meters)+'_'+str(iterations)+'_px.npy',x_zhang_R)
np.save('x_zhang_L_'+str(meters)+'_'+str(iterations)+'_px.npy',x_zhang_L)

(5, 117, 1024, 3)
(117, 1024, 4)
(5, 117, 1024, 3)
World points:  (5, 1, 119808, 3)
Image points:  (5, 117, 1024, 2)


In [249]:
"""
fig, (ax1,ax2) = plt.subplots(1,2)

for i in range(0,80,1):
    ax2.plot(x_zhang_R[i,:,:,0],x_zhang_R[i,:,:,1], color = 'r',ls = "None", marker = ".")
    ax1.plot(x_zhang_L[i,:,:,0],x_zhang_L[i,:,:,1], color = 'r',ls = "None", marker = ".")
ax1.set_aspect(1)
ax2.set_aspect(1)
ax2.set_xlim([0,1936])
ax1.set_xlim([0,1936])
ax2.set_ylim([0,1216])
ax1.set_ylim([0,1216])
ax1.grid()
ax2.grid()
plt.show()
"""

'\nfig, (ax1,ax2) = plt.subplots(1,2)\n\nfor i in range(0,80,1):\n    ax2.plot(x_zhang_R[i,:,:,0],x_zhang_R[i,:,:,1], color = \'r\',ls = "None", marker = ".")\n    ax1.plot(x_zhang_L[i,:,:,0],x_zhang_L[i,:,:,1], color = \'r\',ls = "None", marker = ".")\nax1.set_aspect(1)\nax2.set_aspect(1)\nax2.set_xlim([0,1936])\nax1.set_xlim([0,1936])\nax2.set_ylim([0,1216])\nax1.set_ylim([0,1216])\nax1.grid()\nax2.grid()\nplt.show()\n'

In [250]:
""" world_pts_arr = np.load('world_pts_arr_2_80.npy')
x_zhang_R = np.load('x_zhang_R_2_80.npy')
x_zhang_L = np.load('x_zhang_L_2_80.npy') """

" world_pts_arr = np.load('world_pts_arr_2_80.npy')\nx_zhang_R = np.load('x_zhang_R_2_80.npy')\nx_zhang_L = np.load('x_zhang_L_2_80.npy') "

In [251]:
def spot_center_coord(cx0,cy0,pixelGridX,pixelGridY):
    cx0_max = np.where((cx0 < pixelGridX))
    cx0_min = np.where((cx0 > pixelGridX))
    #print(cx0_max[0][0])
    #print(cx0_min[0][-1])
    cy0_max = np.where((cy0 < pixelGridY))
    cy0_min = np.where((cy0 > pixelGridY))
    #print(cy0_max[0][0])
    #print(cy0_min[0][-1])

    return cx0_max[0][0],cx0_min[0][-1],cy0_max[0][0],cy0_min[0][-1]


def findcircle(x_arr):
    """
    Based on the image coordinates, and knowing it forms a square/rectangle, finds the centroid and radius

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    Returns
    -------
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius
    """  
    
    Ax  = x_arr[0,0]
    Ay  = x_arr[0,1]
    Bx  = x_arr[int(x_arr.shape[0]/4),0]
    By  = x_arr[int(x_arr.shape[0]/4),1]
    Cx  = x_arr[int(x_arr.shape[0]/2),0]
    Cy  = x_arr[int(x_arr.shape[0]/2),1]
    Dx  = x_arr[int((x_arr.shape[0]/4*3)-1),0]
    Dy  = x_arr[int((x_arr.shape[0]/4*3)-1),1]


    cx0 = np.sum(x_arr[:,0])/np.sum(x_arr[:,0].shape[0])
    cy0 = np.sum(x_arr[:,1])/np.sum(x_arr[:,1].shape[0])

    r0 = np.abs((Dy-By)/2)
    
    return cx0,cy0,r0

def createcircle(x_arr,nrpoints = 100):
    """
    create the circle, using the function findcircle()

    Parameters
    ----------
    x_arr:  np.array
        image coordinates
    nrpoints: integer
        number of points that compose the circle
    Returns
    -------
    circle_x:    np.array
        points that define the circle in the cx0 coordinate
    circle_y:    np.array
        points that define the circle in the cy0 coordinate
    cx0:    np.float
        centroid x
    cy0:    np.float
        centroid y    
    r0 : np.float
        circle radius    
    """    

    cx0,cy0,r0 = findcircle(x_arr)

    theta = np.linspace(0, 2*np.pi, nrpoints)
    x = r0*np.cos(theta)
    y = r0*np.sin(theta)

    circle_x = x+cx0
    circle_y = y+cy0


    return circle_x, circle_y,cx0,cy0,r0

In [252]:
"""
NMI : 2021-11-11 
Minimum Volume Enclosing Ellipsoids, see e.g.
NIMA MOSHTAGH : MINIMUM VOLUME ENCLOSING ELLIPSOIDS
or 
Linus Källberg : Minimum_Enclosing_Balls_and_Ellipsoids (Thesis)
"""
from warnings import warn

from numpy import pi
from numpy import sqrt
from numpy import arccos
from numpy import dot, outer
from numpy import diag, transpose
from numpy import append
from numpy import asarray
from numpy import ones
from numpy import argmax

from numpy.linalg import inv
from numpy.linalg import norm
from numpy.linalg import eig


def mvee( data, tolerance=1e-4, maxcnt=10000 ):
    """
    param data: list of xy data points
    param tolerance: termination condition for iterative approximation
    param maxcnt: maximum number of iterations
    type data: iterable of float
    type tolerance: float
    return: (offset, semiaxis, angle)
    return type: ( (float, float), (float, float), float )
    """
    locdata = asarray( data )
    N = len( locdata )
    if not locdata.shape == ( N, 2):
        raise ValueError ( " data must be of shape( n, 2 )" )
    if tolerance >= 1 or tolerance <= 0:
        raise ValueError (" 0 < tolerance < 1 required")
    if not isinstance( maxcnt, int ):
        raise TypeError
    if not maxcnt > 0:
        raise ValueError
    count = 1
    err = 1
    d = 2
    d1 = d + 1
    u = ones( N ) / N
    P = transpose( locdata )
    Q = append( P, ones( N ) ).reshape( 3, -1 )
    while ( err > tolerance):
        X = dot( Q, dot( diag( u ), transpose( Q ) ) )
        M = diag( 
            dot( 
                transpose( Q ),
                dot(
                    inv( X ),
                    Q
                )
            )
        )
        maximum = max( M )
        j = argmax( M )
        step_size = ( maximum - d1 ) / ( d1 * ( maximum - 1 ) )
        new_u = ( 1 - step_size ) * u
        new_u[ j ] += step_size
        err = norm( new_u - u )
        count = count + 1
        u = new_u
        if count > maxcnt:
            warn(
                "Process did not converge in {} steps".format(
                    count - 1
                ),
                UserWarning
            )
            break
    U = diag( u )
    c = dot( P,  u )
    A = inv(
        dot(
            P,
            dot( U, transpose( P ) )
        ) - outer( c, c )
    ) / d
    E, V = eig( A )
    phiopt = arccos( V[ 0, 0 ] )
    if V[ 0, 1 ] < 0: 
        phiopt = 2 * pi - phiopt
    ### cw vs ccw and periodicity of pi
    phiopt = -phiopt % pi
    sol =  (  c, sqrt( 1.0 / E ), phiopt)
    return sol

In [253]:
x0_L = x_zhang_L[:,:,:,0]
y0_L = x_zhang_L[:,:,:,1]


x0_R = x_zhang_R[:,:,:,0]
y0_R = x_zhang_R[:,:,:,1]

### Find the center of the circle(point)

In [254]:
# Generate pixel grid
pixelGridX  = (np.arange(0,image_size[0])) 
pixelGridY  = (np.arange(0,image_size[1]))
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

cx0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cy0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
r0_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))
cx0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
cy0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))
r0_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))

#
x0_cL = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
y0_cL = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],gridSize))
x0_cR = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))
y0_cR = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],gridSize))


for chess_i in range(x0_L.shape[0]):
    for point_i in range(x0_L.shape[1]):

        #circle definition 
        # multiply by pixel_width to get the results in mm, not in pixels
        x0_cL[chess_i,point_i,:],y0_cL[chess_i,point_i,:],cx0_L[chess_i,point_i,:],cy0_L[chess_i,point_i,:],r0_L[chess_i,point_i,:] = createcircle(x_zhang_L[chess_i,point_i,:],gridSize)
        x0_cR[chess_i,point_i,:],y0_cR[chess_i,point_i,:],cx0_R[chess_i,point_i,:],cy0_R[chess_i,point_i,:],r0_R[chess_i,point_i,:] = createcircle(x_zhang_R[chess_i,point_i,:],gridSize)

## Find the centroids of every ellipse and circle using the moment method

In [255]:
pixelGridX  = (np.arange(0,image_size[0])) 
pixelGridY  = (np.arange(0,image_size[1])) 
pixelGridXX, pixelGridYY = np.meshgrid(pixelGridX,pixelGridY)

world_pts_arr,x_zhang_R_px,x_zhang_L_px, P_R, P_L = create_ideal_chessboard_pixel(xrange_5,yrange_5,zrange_5)

World points:  (5, 1, 117, 3)
Image points:  (5, 117, 2)


In [256]:
np.save("world_pts_arr_f_"+str(meters)+"_"+str(iterations)+"_px.npy",world_pts_arr)

In [257]:
# get for 512 per spot
gridSize = 1024

world_pts_arr,x_zhang_R,x_zhang_L,P_R,P_L = create_1024_point_chess(xrange_5,yrange_5,zrange_5)


# create the arrays to store the centroids
momentL = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
momentR = np.zeros((x0_R.shape[0],x0_R.shape[1],2))

#for each chessboard
for chess_i in range(x0_L.shape[0]):
    for point_i in range(x0_L.shape[1]):
        momentL[chess_i,point_i,0] = np.sum(x0_L[chess_i,point_i,:])/np.sum(x0_L.shape[2])
        momentL[chess_i,point_i,1] = np.sum(y0_L[chess_i,point_i,:])/np.sum(y0_L.shape[2])

        momentR[chess_i,point_i,0] = np.sum(x0_R[chess_i,point_i,:])/np.sum(x0_R.shape[2])
        momentR[chess_i,point_i,1] = np.sum(y0_R[chess_i,point_i,:])/np.sum(y0_R.shape[2])

(5, 117, 1024, 3)
(117, 1024, 4)
(5, 117, 1024, 3)
World points:  (5, 1, 119808, 3)
Image points:  (5, 117, 1024, 2)


### Difference between original point without spot and centroiding the spot

In [258]:
np.sum(momentL[:,:,:]-(x_zhang_L_px[:,:,:]))

0.8560404777526855

In [259]:
np.sum(momentR[:,:,:]-(x_zhang_R_px[:,:,:]))

0.07620954513549805

### Find max and mim values that create boundaries in the ellipse and plot the center of the ellipse 
Also, meshgrid is added to the ellipse, representing the pixels

In [260]:
x_L_lst = []
y_L_lst = []
for chess_b_i in range(x0_L.shape[0]):
    x_L_pts = []
    y_L_pts = []
    for point in range(x0_L.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x_L = np.max(x_zhang_L[chess_b_i,point,:,0])
        max_pt_y_L = np.max(x_zhang_L[chess_b_i,point,:,1])
        min_pt_x_L = np.min(x_zhang_L[chess_b_i,point,:,0])
        min_pt_y_L = np.min(x_zhang_L[chess_b_i,point,:,1])

        pts = np.array([[max_pt_x_L,max_pt_y_L],[max_pt_x_L,min_pt_y_L],[min_pt_x_L,min_pt_y_L],[min_pt_x_L,max_pt_y_L]])
        cmin_pxl_L = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max_L,cx0min_L,cy0max_L,cy0min_L  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_L[pts_i,0] = (cx0min_L)
            cmin_pxl_L[pts_i,1] = (cy0min_L)

        x_L_pts.append(np.arange(cmin_pxl_L[3,0],cmin_pxl_L[0,0]+1,1))
        y_L_pts.append(np.arange(cmin_pxl_L[1,1],cmin_pxl_L[0,1]+1,1))
        xx,yy = np.meshgrid(x_L_pts[point],y_L_pts[point])

        #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")


    x_L_lst.append(x_L_pts)
    y_L_lst.append(y_L_pts)
#ax.set_aspect(1)    
#plt.show()

IndexError: index -1 is out of bounds for axis 0 with size 0

In [ ]:
x_R_lst = []
y_R_lst = []
for chess_b_i in range(x0_R.shape[0]):
    x_R_pts = []
    y_R_pts = []
    for point in range(x0_R.shape[1]):
        #ax.plot(x0_L[chess_b_i,point,:],y0_L[chess_b_i,point,:])
        max_pt_x_R = np.max(x_zhang_R[chess_b_i,point,:,0])
        max_pt_y_R = np.max(x_zhang_R[chess_b_i,point,:,1])
        min_pt_x_R = np.min(x_zhang_R[chess_b_i,point,:,0])
        min_pt_y_R = np.min(x_zhang_R[chess_b_i,point,:,1])

        pts = np.array([[max_pt_x_R,max_pt_y_R],[max_pt_x_R,min_pt_y_R],[min_pt_x_R,min_pt_y_R],[min_pt_x_R,max_pt_y_R]])
        cmin_pxl_R = np.zeros((pts.shape[0],pts.shape[1]))
        for pts_i,pts_d in enumerate(pts):
            cx0max_R,cx0min_R,cy0max_R,cy0min_R  = spot_center_coord(pts_d[0],pts_d[1],pixelGridX,pixelGridY)

            # max and min points in (x,y) coordinates
            cmin_pxl_R[pts_i,0] = (cx0min_R)
            cmin_pxl_R[pts_i,1] = (cy0min_R)

        x_R_pts.append(np.arange(cmin_pxl_R[3,0],cmin_pxl_R[0,0]+1,1))
        y_R_pts.append(np.arange(cmin_pxl_R[1,1],cmin_pxl_R[0,1]+1,1))
        xx,yy = np.meshgrid(x_R_pts[point],y_R_pts[point])

        #ax.plot(xx,yy, color = 'g',ls = "None", marker = "*")


    x_R_lst.append(x_R_pts)
    y_R_lst.append(y_R_pts)
#ax.set_aspect(1)    
#plt.show()

### Find the ellipses center, semi-major and semi-minor

In [ ]:
X_R = np.array(x_zhang_R[:,:,:,:])
X_L = np.array(x_zhang_L[:,:,:,:])

xy_R_arr = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],2))
ab_R_arr = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],2))
theta_arr_R = np.zeros((x_zhang_R.shape[0],x_zhang_R.shape[1],1))

xy_L_arr = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],2))
ab_L_arr = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],2))
theta_arr_L = np.zeros((x_zhang_L.shape[0],x_zhang_L.shape[1],1))

# centroid array
for chess_i in range(x_zhang_L.shape[0]):
    for point_i in range(x_zhang_L.shape[1]):
        xy_L_arr[chess_i,point_i,:], ab_L_arr[chess_i,point_i,:],theta_arr_R[chess_i,point_i,:] = mvee(X_L[chess_i,point_i,:,:])
        xy_R_arr[chess_i,point_i,:], ab_R_arr[chess_i,point_i,:],theta_arr_L[chess_i,point_i,:] = mvee(X_R[chess_i,point_i,:,:])


#### Check if the difference between the mvee method and the original spot is correct

In [ ]:
np.sum((x_zhang_L_px[:,:,:])-xy_L_arr[:,:,:])
#NOTE: The result is very good

-2.8114183218666042

In [ ]:
np.sum((x_zhang_R_px[:,:,:])-xy_R_arr[:,:,:])
#NOTE: The result is very good

1.9948199207313522

### Do analysis pixel by pixel and check if correlates withthe original ellipse

In [ ]:
def circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,chess,point,subpixel_grid,pixel_width):
       
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)
        
        circ_L = np.sqrt((xx_pixel_L-cx0_L[chess,point,:])**2 + (yy_pixel_L-cy0_L[chess,point,:])**2) <= r0_L[chess,point,:]
        circ_corners_L = np.where(circ_L == True)
        xxgrid_L = np.array(circ_corners_L).T


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), pixel_width,pixel_width))

    return xxgrid_L,lstL,ptchsL

In [ ]:
def ellipsediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,xc_arr_L,yc_arr_L,a_arr_L,b_arr_L,theta_arr,chess,point,subpixel_grid,pixel_width):
    """
    Based on the parameters fed to the function, it discretizes the ellipse returning the occupied percentage of each pixel by the ellipse
    ----------
    X_crns: np.array
        Coordinate mesh grid on the x axis
    Y_crns:    np.array   
        Coordinate mesh grid on the y axis 
    x_subgrid:    np.array
        x-axis grid for the subgrid portion
    y_subgrid:    np.array
        y-axis grid for the subgrid portion
    xc_arr
    yc_arr
    a_arr
    b_arr
    theta_arr
    chess
    point
    subpixel_grid
    pixel_width
    
    Returns    
    -------    
    world_pts_arr : np.array
        world coordinates
    x_zhang_R:    np.array
        right image coordinates
    x_zhang_L:    np.array
        Left image coordinates
    P_L : np.array
        Left camera Projection matrix
    P_R : np.array
        Right camera Projection matrix
                
    """
    
    lstL = []
    ptchsL = []

    for i in range(X_crns_L.flatten().shape[0]):
        cx0max_L = X_crns_L.flatten()[i]
        cy0max_L = Y_crns_L.flatten()[i]

        # translate the left bottom corner
        x_pixel_L = cx0max_L+xgrid_L
        y_pixel_L = cy0max_L+ygrid_L
        # create sub-pixel grid
        xx_pixel_L, yy_pixel_L = np.meshgrid(x_pixel_L,y_pixel_L)

        x_L, y_L = xx_pixel_L.flatten(), yy_pixel_L.flatten()
                
        # calculate whether a point is inside the ellipse or not
        cos_angle = np.cos(np.radians(180.-theta_arr*180/np.pi))
        sin_angle = np.sin(np.radians(180.-theta_arr*180/np.pi))

        x_c_L = x_L - xc_arr_L
        y_c_L = y_L - yc_arr_L

        xct_L = x_c_L * cos_angle - y_c_L * sin_angle
        yct_L = x_c_L * sin_angle + y_c_L * cos_angle 

        rad_cc = (xct_L**2/(a_arr_L)**2) + (yct_L**2/(b_arr_L)**2)

        # set the 
        xxgrid_L = np.where(rad_cc <= 1.)[0]


        lstL.append((xxgrid_L.shape[0]/(subpixel_grid**2))*100)

        ptchsL.append(Rectangle((cx0max_L,cy0max_L), 1,1))
    
    return xxgrid_L,lstL,ptchsL 

In [ ]:

from matplotlib.path import Path


import warnings
warnings.filterwarnings("error")

centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
xgrid_L = np.linspace(0,1,subpixel_grid)
ygrid_L = np.linspace(0,1,subpixel_grid)

chess_pts_L = []
chess_ptchs_L = []

centroid_L_arr = np.zeros((x0_L.shape[0],x0_L.shape[1],2))
# iterate the chessboards

for chess_i in range(x0_L.shape[0]):

    lst_pts_L = []
    ptchs_pts_L = []
    # runs each point in the chessboard
    for point_i in range(x0_L.shape[1]):
        
        lstL = []
        ptchsL = []
        
        # meshgrid for the pixel
        [X_crns_L,Y_crns_L] = np.meshgrid(x_L_lst[chess_i][point_i],y_L_lst[chess_i][point_i])

        if(chess_i % 5 == 0):
            xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,chess_i,point_i,subpixel_grid,1)

        else:
            xxgrid_L,lstL,ptchsL  = ellipsediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,xy_L_arr[chess_i,point_i,0],xy_L_arr[chess_i,point_i,1],ab_L_arr[chess_i,point_i,0],ab_L_arr[chess_i,point_i,1],theta_arr_L[chess_i,point_i,:],chess_i,point_i,subpixel_grid,1)
        # if the max and min values as 0 it means the point was not correctly identified
        if(np.max(lstL) == 0 and np.min(lstL) == 0):
            xxgrid_L,lstL,ptchsL = circlediscretization(X_crns_L,Y_crns_L,xgrid_L,ygrid_L,cx0_L,cy0_L,r0_L,chess_i,point_i,subpixel_grid,1)
                                                        

        centroid_L_x = np.sum(np.multiply(X_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5
        centroid_L_y = np.sum(np.multiply(Y_crns_L.flatten(),lstL))/(np.sum(lstL)) + 0.5
        centroid_L_arr[chess_i,point_i,:]  = (centroid_L_x,centroid_L_y)

        lst_pts_L.append(lstL)
        ptchs_pts_L.append(ptchsL)            

    chess_ptchs_L.append(ptchs_pts_L)
    chess_pts_L.append(lst_pts_L)

In [ ]:
np.sum(centroid_L_arr[:,:,:] - (x_zhang_L_px[:,:,:]))

10.378232606127206

In [ ]:

warnings.filterwarnings("error")

centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
xgrid_R = np.linspace(0,1,subpixel_grid)
ygrid_R = np.linspace(0,1,subpixel_grid)

chess_pts_R = []
chess_ptchs_R = []

centroid_R_arr = np.zeros((x0_R.shape[0],x0_R.shape[1],2))
# iterate the chessboards

for chess_i in range(x0_R.shape[0]):

    lst_pts_R = []
    ptchs_pts_R = []
    # runs each point in the chessboard
    for point_i in range(x0_R.shape[1]):
        
        lstR = []
        ptchsR = []
        
        # meshgrid for the pixel
        [X_crns_R,Y_crns_R] = np.meshgrid(x_R_lst[chess_i][point_i],y_R_lst[chess_i][point_i])

        if(chess_i % 5 == 0):
            xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,chess_i,point_i,subpixel_grid,1)

        else:
            xxgrid_R,lstR,ptchsR  = ellipsediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,xy_R_arr[chess_i,point_i,0],xy_R_arr[chess_i,point_i,1],ab_R_arr[chess_i,point_i,0],ab_R_arr[chess_i,point_i,1],theta_arr_R[chess_i,point_i,:],chess_i,point_i,subpixel_grid,1)
        # if the max and min values as 0 it means the point was not correctly identified
        if(np.max(lstR) == 0 and np.min(lstR) == 0):
            xxgrid_R,lstR,ptchsR = circlediscretization(X_crns_R,Y_crns_R,xgrid_R,ygrid_R,cx0_R,cy0_R,r0_R,chess_i,point_i,subpixel_grid,1)
                                                        

        centroid_R_x = np.sum(np.multiply(X_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5
        centroid_R_y = np.sum(np.multiply(Y_crns_R.flatten(),lstR))/(np.sum(lstR)) + 0.5
        centroid_R_arr[chess_i,point_i,:]  = (centroid_R_x,centroid_R_y)

        lst_pts_R.append(lstR)
        ptchs_pts_R.append(ptchsR)            

    chess_ptchs_R.append(ptchs_pts_R)
    chess_pts_R.append(lst_pts_R)

In [ ]:
np.sum(centroid_R_arr[:,:,:] - (x_zhang_R_px[:,:,:]))

17.457559232870913

In [ ]:
np.save(f"centroid_R_arr_{meters}_{iterations}_px.npy",centroid_R_arr)
np.save(f"centroid_L_arr_{meters}_{iterations}_px.npy",centroid_L_arr)

In [ ]:
centroid_R_arr

array([[[  36.94407338,  112.43979212],
        [  66.9790527 ,  112.43971803],
        [  97.01391866,  112.43974947],
        ...,
        [ 337.29118308,  352.7052612 ],
        [ 367.32541525,  352.70522969],
        [ 397.35941529,  352.705282  ]],

       [[ 280.72510872,   34.52624774],
        [ 280.72500237,   65.52492663],
        [ 280.72507714,   96.52352572],
        ...,
        [ 157.41360613,  360.41555318],
        [ 157.41375234,  389.5398324 ],
        [ 157.41378523,  418.66363546]],

       [[ 365.40829321,  127.57626401],
        [ 360.89091062,  144.41313608],
        [ 356.31942446,  161.51778993],
        ...,
        [  69.03994582,  308.2019308 ],
        [  61.76726551,  327.86168777],
        [  54.36864483,  347.85274862]],

       ...,

       [[1474.07735823,  844.94621813],
        [1477.86043144,  867.16390897],
        [1481.70515969,  889.71598749],
        ...,
        [1266.71098801, 1083.15811888],
        [1269.13280734, 1109.10246079],
        [

In [ ]:
x_zhang_R_px

array([[[  36.94198 ,  112.43686 ],
        [  66.97611 ,  112.43686 ],
        [  97.01024 ,  112.43686 ],
        ...,
        [ 337.28326 ,  352.7099  ],
        [ 367.3174  ,  352.7099  ],
        [ 397.35153 ,  352.7099  ]],

       [[ 280.71274 ,   34.526287],
        [ 280.71274 ,   65.524864],
        [ 280.71274 ,   96.523445],
        ...,
        [ 157.41684 ,  360.413   ],
        [ 157.41684 ,  389.5409  ],
        [ 157.41684 ,  418.66876 ]],

       [[ 365.40576 ,  127.57054 ],
        [ 360.89417 ,  144.41588 ],
        [ 356.31445 ,  161.51538 ],
        ...,
        [  69.04121 ,  308.19498 ],
        [  61.766006,  327.8654  ],
        [  54.372093,  347.85675 ]],

       ...,

       [[1474.0748  ,  844.94464 ],
        [1477.8638  ,  867.16095 ],
        [1481.71    ,  889.71246 ],
        ...,
        [1266.7136  , 1083.1594  ],
        [1269.1311  , 1109.1016  ],
        [1271.588   , 1135.467   ]],

       [[1244.5151  , 1088.4294  ],
        [1246.5853  , 1071.